In [1]:
from pathlib import Path

import pandas as pd
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

In [2]:
JAN_FHV_DATA_PATH = "D:\\data\\nyc-green-taxi\\fhv_tripdata_2021-01.parquet"
FEB_FHV_DATA_PATH = "D:\\data\\nyc-green-taxi\\fhv_tripdata_2021-02.parquet"

# Q1. Downloading the data
We'll use the same NYC taxi dataset, but instead of "Green Taxi Trip Records", we'll use "For-Hire Vehicle Trip Records".

Download the data for January and February 2021.

Note that you need "For-Hire Vehicle Trip Records", not "High Volume For-Hire Vehicle Trip Records".

Read the data for January. How many records are there?

In [3]:
janfhv_df = pd.read_parquet(JAN_FHV_DATA_PATH)
janfhv_df

,dispatching_base_num,pickup_datetime,dropOff_datetime,PUlocationID,DOlocationID,SR_Flag,Affiliated_base_number
0,B00009,2021-01-01 00:27:00,2021-01-01 00:44:00,NaN,NaN,None,B00009
1,B00009,2021-01-01 00:50:00,2021-01-01 01:07:00,NaN,NaN,None,B00009
2,B00013,2021-01-01 00:01:00,2021-01-01 01:51:00,NaN,NaN,None,B00013
3,B00037,2021-01-01 00:13:09,2021-01-01 00:21:26,NaN,72.0,None,B00037
4,B00037,2021-01-01 00:38:31,2021-01-01 00:53:44,NaN,61.0,None,B00037
...,...,...,...,...,...,...,...
1154107,B03266,2021-01-31 23:43:03,2021-01-31 23:51:48,7.0,7.0,None,B03266
1154108,B03284,2021-01-31 23:50:27,2021-02-01 00:48:03,44.0,91.0,None,
1154109,B03285,2021-01-31 23:13:46,2021-01-31 23:29:58,171.0,171.0,None,B03285
1154110,B03285,2021-01-31 23:58:03,2021-02-01 00:17:29,15.0,15.0,None,B03285


# Q2. Computing duration
Now let's compute the duration variable. It should contain the duration of a ride in minutes.

What's the average trip duration in January?

In [4]:
janfhv_df['duration'] = janfhv_df.dropOff_datetime - janfhv_df.pickup_datetime
janfhv_df.duration = janfhv_df.duration.apply(lambda td: td.total_seconds() / 60)
janfhv_df.duration.mean()

19.167224093791006

In [5]:
janfhv_df = janfhv_df[(janfhv_df.duration >= 1) & (janfhv_df.duration <= 60)]

# Q3. Missing values
The features we'll use for our model are the pickup and dropoff location IDs.

But they have a lot of missing values there. Let's replace them with "-1".

What's the fractions of missing values for the pickup location ID? I.e. fraction of "-1"s after you filled the NAs.

In [6]:
categorical = ['PUlocationID', 'DOlocationID']
print(janfhv_df.PUlocationID.isna().mean())
janfhv_df[categorical] = janfhv_df[categorical].fillna(-1).astype(str)
janfhv_df[categorical]

0.8352732770722617


C:\Users\mailt\AppData\Local\Temp\ipykernel_16448\972666464.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  janfhv_df[categorical] = janfhv_df[categorical].fillna(-1).astype(str)


,PUlocationID,DOlocationID
0,-1.0,-1.0
1,-1.0,-1.0
3,-1.0,72.0
4,-1.0,61.0
5,-1.0,71.0
...,...,...
1154107,7.0,7.0
1154108,44.0,91.0
1154109,171.0,171.0
1154110,15.0,15.0


# Q4. One-hot encoding
Let's apply one-hot encoding to the pickup and dropoff location IDs. We'll use only these two features for our model.

Turn the dataframe into a list of dictionaries
Fit a dictionary vectorizer
Get a feature matrix from it
What's the dimensionality of this matrix? (The number of columns).

In [7]:
dv = DictVectorizer()

train_dicts = janfhv_df[categorical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)
X_train.shape

(1109826, 525)

# Q5. Training a model
Now let's use the feature matrix from the previous step to train a model.

Train a plain linear regression model with default parameters
Calculate the RMSE of the model on the training data
What's the RMSE on train?

In [8]:
target = 'duration'
y_train = janfhv_df[target].values

lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_train)

mean_squared_error(y_train, y_pred, squared=False)

10.5285194339756

# Q6. Evaluating the model
Now let's apply this model to the validation dataset (Feb 2021).

What's the RMSE on validation?

In [9]:
def preprocess_data(data_path: Path) -> pd.DataFrame:
    if not data_path.endswith('.parquet'):
        raise ValueError("Only .parquet files are supported")
    else:
        df = pd.read_parquet(data_path)
        
        df['duration'] = df.dropOff_datetime - df.pickup_datetime
        df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)
        df = df[(df.duration >= 1) & (df.duration <= 60)]
        
        categorical = ['PUlocationID', 'DOlocationID']
        df[categorical] = df[categorical].fillna(-1).astype(str)
        
        return df
    
    
febfhv_df = preprocess_data(FEB_FHV_DATA_PATH)
febfhv_df

,dispatching_base_num,pickup_datetime,dropOff_datetime,PUlocationID,DOlocationID,SR_Flag,Affiliated_base_number,duration
1,B00021,2021-02-01 00:55:40,2021-02-01 01:06:20,173.0,82.0,None,B00021,10.666667
2,B00021,2021-02-01 00:14:03,2021-02-01 00:28:37,173.0,56.0,None,B00021,14.566667
3,B00021,2021-02-01 00:27:48,2021-02-01 00:35:45,82.0,129.0,None,B00021,7.950000
4,B00037,2021-02-01 00:12:50,2021-02-01 00:26:38,-1.0,225.0,None,B00037,13.800000
5,B00037,2021-02-01 00:00:37,2021-02-01 00:09:35,-1.0,61.0,None,B00037,8.966667
...,...,...,...,...,...,...,...,...
1037687,B03282,2021-02-28 23:01:16,2021-02-28 23:14:48,-1.0,31.0,None,B01717,13.533333
1037688,B03282,2021-02-28 23:36:10,2021-02-28 23:47:38,-1.0,169.0,None,B01717,11.466667
1037689,B03285,2021-02-28 23:18:36,2021-02-28 23:43:59,28.0,171.0,None,B03285,25.383333
1037690,B03285,2021-02-28 23:26:34,2021-02-28 23:44:37,16.0,252.0,None,B03285,18.050000


In [10]:
y_val = febfhv_df[target].values
val_dicts = febfhv_df[categorical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

y_pred = lr.predict(X_val)
mean_squared_error(y_val, y_pred, squared=False)

11.014285894450383